ANALISIS RAPIDO

In [1]:
import sys
from pathlib import Path
import pandas as pd

# Agregamos la carpeta src al path para poder importar
path_inicio = Path.cwd().parent/"src"
sys.path.append(str(path_inicio))
from utilidades.constantes import  MOSTRADOR_SALIDA_PATH, FACTURACION_SALIDA_PATH, ARCHIVO_UNIFICADO_PATH, TOTAL_FINAL_PATH


In [2]:

df_mostrador = pd.read_csv(MOSTRADOR_SALIDA_PATH, encoding='utf-8-sig')
df_facturacion = pd.read_csv(FACTURACION_SALIDA_PATH, encoding='utf-8-sig')
df_unificado = pd.read_csv(ARCHIVO_UNIFICADO_PATH, encoding='utf-8-sig')
df_total = pd.read_csv(TOTAL_FINAL_PATH, encoding='utf-8-sig')

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40300\1778513677.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mostrador = pd.read_csv(MOSTRADOR_SALIDA_PATH, encoding='utf-8-sig')
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_40300\1778513677.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_unificado = pd.read_csv(ARCHIVO_UNIFICADO_PATH, encoding='utf-8-sig')


In [32]:
# Convertir las fechas a datetime
df_mostrador['Fecha'] = pd.to_datetime(df_mostrador['Fecha'])
df_facturacion['fecha_emision'] = pd.to_datetime(df_facturacion['fecha_emision'])

# Filtrar año 2024 y agregar columna 'mes'
df_mostrador_2024 = df_mostrador[df_mostrador['Fecha'].dt.year == 2024].copy()
df_mostrador_2024['mes'] = df_mostrador_2024['Fecha'].dt.month

df_facturacion_2024 = df_facturacion[df_facturacion['fecha_emision'].dt.year == 2024].copy()
df_facturacion_2024['mes'] = df_facturacion_2024['fecha_emision'].dt.month

# Sumar ventas por mes
ventas_mostrador_2024 = df_mostrador_2024.groupby('mes')['TotalVenta'].sum()
ventas_facturacion_2024 = df_facturacion_2024.groupby('mes')['importe_total'].sum()

# Sumar ambos resultados por mes
ventas_mensuales_2024 = ventas_mostrador_2024.add(ventas_facturacion_2024, fill_value=0).sort_index()
ventas_mensuales_2024

mes
1      41626591
2      61937996
3     117385793
4     104687537
5     168259585
6     110520225
7     159160635
8     179654262
9     211066304
10    226609483
11    274872169
12    486447015
dtype: int64

In [3]:
# Filtrar solo ventas del año 2024 y sumar la columna 'monto'
total_ventas_unificado_2024 = df_unificado[df_unificado['anio'] == 2024]['monto'].sum()
total_ventas_unificado_2024

np.int64(1851201775)

In [5]:
# Filtrar solo ventas del año 2023 y sumar la columna 'monto'
total_ventas_unificado_2023 = df_unificado[df_unificado['anio'] == 2023]['monto'].sum()
total_ventas_unificado_2023

np.int64(820087754)

In [4]:
# Filtrar filas del año 2024 con montos negativos y sumar
monto_negativos_2024 = df_unificado[(df_unificado['anio'] == 2024) & (df_unificado['monto'] < 0)]['monto'].sum()
monto_negativos_2024

np.int64(-147399741)

In [3]:
# Primero, separamos las columnas del único string en df_total
df_total_split = df_total['Fecha de Emisión;Tipo de Comprobante;Imp. Total'].str.split(';', expand=True)
df_total_split.columns = ['fecha_emision', 'tipo_comprobante', 'imp_total']

# Convertimos tipo_comprobante e imp_total a numérico
df_total_split['tipo_comprobante'] = pd.to_numeric(df_total_split['tipo_comprobante'], errors='coerce')
df_total_split['imp_total'] = pd.to_numeric(df_total_split['imp_total'], errors='coerce')

# Sumar ventas, restando los comprobantes tipo 3 y 8
total_ventas_final = (
    df_total_split.apply(
        lambda row: -row['imp_total'] if row['tipo_comprobante'] in [3, 8] else row['imp_total'],
        axis=1
    ).sum()
)
total_ventas_final

np.float64(1781075504.98)